In [ ]:
# Load data
data = pd.read_csv("../data/data_dol_val.csv")

# Convert 'data' to datetime and set it as the index
data['data'] = pd.to_datetime(data['data'])
data = data.set_index('data')

# Aggregate avg_preco_venda by product (ignoring states)
product_prices = (
    data.groupby('produto')[['avg_preco_venda']]
    .resample('M')
    .mean()
    .reset_index()
)

# Aggregate dol_val separately (monthly mean)
usd_prices = (
    data[['dol_val']]
    .resample('M')
    .mean()
    .reset_index()
)

# Create figure for avg_preco_venda
fig = px.line(
    product_prices,
    x='data',
    y='avg_preco_venda',
    color='produto',  # Different colors for each product
    title="Média de Preço de Venda por Produto e Taxa de Câmbio (USD)",
    labels={'data': 'Data', 'avg_preco_venda': 'Preço Médio de Venda', 'produto': 'Produto'},
    hover_data={'produto': True}
)

# Convert px figure to go.Figure()
fig = go.Figure(fig)

# Add USD Exchange Rate as a separate trace
fig.add_trace(
    go.Scatter(
        x=usd_prices['data'],
        y=usd_prices['dol_val'],
        mode='lines',
        name='USD Exchange Rate',
        line=dict(color='yellow', width=2, dash='solid')  # Continuous black line
    )
)

# Enable **multiple hover windows for the same date**
fig.update_layout(hovermode="x unified")

# Add a vertical dashed line for March 2020
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0="2020-03-01", x1="2020-03-01",
        y0=product_prices['avg_preco_venda'].min(),
        y1=product_prices['avg_preco_venda'].max(),
        line=dict(color="red", width=2, dash="dash")
    )
)

# Add annotation for the event
fig.add_annotation(
    x="2020-03-01",
    y=product_prices['avg_preco_venda'].max(),
    text="Pandemia COVID-19",
    showarrow=True,
    arrowhead=2,
    font=dict(size=12, color="red"),
    ax=50,
    ay=-40
)

# Show the interactive plot
fig.show()
